In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 814 (delta 3), reused 8 (delta 3), pack-reused 804
Receiving objects: 100% (814/814), 3.38 MiB | 12.32 MiB/s, done.
Resolving deltas: 100% (531/531), done.
Checking out files: 100% (39/39), done.


In [6]:
!ls

Dockerfile  hubconf.py	      runs		    train_batch2.jpg
LICENSE     inference	      test.py		    tutorial.ipynb
README.md   labels.png	      test_batch0_gt.jpg    utils
config	    models	      test_batch0_pred.jpg  weights
convertor   nohup.out	      train.py		    yolov5
data	    pretrained	      train_batch0.jpg	    yolov5-train.ipynb
detect.py   requirements.txt  train_batch1.jpg


In [4]:
#!mv yolov5/* ./

In [2]:
!ls config

wheat0.yaml  wheat1.yaml  wheat2.yaml  wheat3.yaml  wheat4.yaml  yolov5x.yaml


In [3]:
import sys
sys.path.insert(0, './yolov5train')

In [2]:
#!git clone https://github.com/ultralytics/yolov5

In [3]:
#!mv yolov5/* ./

In [4]:
#!pip install -r requirements.txt

In [4]:
DATA_DIR = '/relevance2-nfs/chec/data/wheat'

In [5]:
df = pd.read_csv(f'{DATA_DIR}/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
from tqdm.auto import tqdm
import shutil as sh
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [6]:
df.head()

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0


In [11]:
index = list(set(df.image_id))

In [12]:
source = 'train'
if True:
    for fold in range(5):
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))

            sh.copy(f"{DATA_DIR}/{source}/{name}.jpg", f'convertor/fold{fold}/images/{path2save}/{name}.jpg')

In [ ]:
!python yolov5train/train.py --img 1024 --batch 24 --epochs 100 --data config/wheat0.yaml \
    --cfg config/yolov5x.yaml --name yolov5x_fold0 \
    --weights pretrained/yolov5x.pt

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=24, bucket='', cache_images=False, cfg='config/yolov5x.yaml', data='config/wheat0.yaml', device='', epochs=100, evolve=False, img_size=[1024], lr0=0.01, multi_scale=False, name='yolov5x_fold0', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='pretrained/yolov5x.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device1 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device2 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device3 _CudaDeviceProperti

In [28]:
!ls weights

best_yolov5x_fold0.pt  last_yolov5x_fold0.pt  yolov5m.pt  yolov5x.pt
download_weights.sh    yolov5l.pt	      yolov5s.pt


In [30]:
import torch
tmp = torch.load('weights/best_yolov5x_fold0.pt')

In [31]:
list(tmp.keys())

['epoch', 'best_fitness', 'training_results', 'model', 'optimizer']

In [32]:
model1 = tmp['model'].cuda()

In [14]:
!python train.py --img 1024 --batch 24 --epochs 50 --data config/wheat3.yaml \
    --cfg config/yolov5x.yaml --name yolov5x_fold3 \
    --weights pretrained/yolov5x.pt

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=24, bucket='', cache_images=False, cfg='config/yolov5x.yaml', data='config/wheat3.yaml', device='', epochs=50, evolve=False, img_size=[1024], multi_scale=False, name='yolov5x_fold3', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='pretrained/yolov5x.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device1 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device2 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device3 _CudaDeviceProperties(name='Te


     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     10/49     19.7G   0.03796    0.1399         0    0.1779      1319      1024
               Class      Images     Targets           P           R      mAP@.5
                 all         672    2.91e+04       0.731       0.943       0.939       0.516

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     11/49     19.7G   0.03695    0.1397         0    0.1766      1580      1024
               Class      Images     Targets           P           R      mAP@.5
                 all         672    2.91e+04       0.737       0.946       0.941       0.523

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     12/49     19.7G   0.03665    0.1373         0    0.1739      1242      1024
               Class      Images     Targets           P           R      mAP@.5
                 all         672    2.91e+04       0.717       0.941       0.935  

In [15]:
!python train.py --img 1024 --batch 24 --epochs 50 --data config/wheat4.yaml \
    --cfg config/yolov5x.yaml --name yolov5x_fold4 \
    --weights pretrained/yolov5x.pt

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=24, bucket='', cache_images=False, cfg='config/yolov5x.yaml', data='config/wheat4.yaml', device='', epochs=50, evolve=False, img_size=[1024], multi_scale=False, name='yolov5x_fold4', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='pretrained/yolov5x.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device1 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device2 _CudaDeviceProperties(name='Tesla P40', total_memory=22919MB)
           device3 _CudaDeviceProperties(name='Te


     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     10/49     19.7G   0.03824    0.1409         0    0.1792      1517      1024
               Class      Images     Targets           P           R      mAP@.5
                 all         672    2.93e+04       0.739       0.942       0.939       0.503

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     11/49     19.7G   0.03688    0.1389         0    0.1758      1789      1024
               Class      Images     Targets           P           R      mAP@.5
                 all         672    2.93e+04        0.71       0.951       0.946       0.524

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
     12/49     19.7G   0.03642    0.1368         0    0.1732      1186      1024
               Class      Images     Targets           P           R      mAP@.5
                 all         672    2.93e+04       0.717        0.95       0.945  